In [63]:
import torch,torchvision
from torchvision import transforms
from torch import nn
from torch import optim
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [64]:
mnist_trainset=torchvision.datasets.MNIST('mnist_data',train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))
mnist_testset=torchvision.datasets.MNIST('mnist_data',train=False,download=True,transform=transforms.Compose([transforms.ToTensor()]))

In [65]:
mnist_valset,mnist_testset=torch.utils.data.random_split(mnist_testset,[int(0.9*len(mnist_testset)),int(0.1*len(mnist_testset))])

In [66]:
train_dataloader=torch.utils.data.DataLoader(mnist_trainset,batch_size=64,shuffle=True)
val_dataloader=torch.utils.data.DataLoader(mnist_valset,batch_size=32,shuffle=False)
test_dataloader=torch.utils.data.DataLoader(mnist_testset,batch_size=32,shuffle=False)

In [67]:
class Model(torch.nn.Module):
        def __init__(self):
            super(Model,self).__init__()
            self.conv_1=nn.Conv2d(1,6,3,stride=1)
            self.conv_2=nn.Conv2d(6,16,3,stride=1)
            self.conv_3=nn.Conv2d(16,120,4,stride=1)
            self.avg_pool2d=nn.AvgPool2d(kernel_size=2,stride=2)
            self.linear_1=nn.Linear(120,84)
            self.linear_2=nn.Linear(84,10)
            self.tanh=nn.Tanh()
            
            
        def forward(self,x):
            x=self.conv_1(x)
            x=self.tanh(x)
            x=self.avg_pool2d(x)
            x=self.conv_2(x)
            x=self.tanh(x)
            x=self.avg_pool2d(x)
            x=self.conv_3(x)
            x=self.tanh(x)
            x=self.avg_pool2d(x)
            x=x.reshape(x.size(0),-1)
            x=self.linear_1(x)
            x=self.tanh(x)
            pred=self.linear_2(x)
            return pred

In [68]:
model=Model()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)    

In [69]:
no_epochs=10
train_loss=list()
val_loss=list()
best_val_loss=1

In [73]:
for epoch in range(no_epochs):
    total_train_loss=0
    total_val_loss=0
    
    model.train()
    #training
    for itr,(image,label) in enumerate(train_dataloader):
        
        optimizer.zero_grad()
        pred=model(image)
        
        loss=criterion(pred,label)
        total_train_loss+=loss.item()
        
        loss.backward()
        optimizer.step()
        
    total_train_loss=total_train_loss/(itr+1)
    train_loss.append(total_train_loss)

In [85]:
#validation
model.eval()
total=0
for itr,(image,label) in enumerate(val_dataloader):
    
        pred=model(image)
    
        loss=criterion(pred,label)
        total_val_loss+=loss.item()
    
        pred=nn.functional.softmax(pred,dim=1)
        for i,p in enumerate(pred):
            if label[i]==torch.max(p.data,0)[1]:
                total=total+1
    
    
accuracy=total/len(mnist_valset)
    
total_val_loss=total_val_loss/(itr+1)
val_loss.append(total_val_loss)
    
if total_val_loss<best_val_loss:
        best_val_loss=total_val_loss
        torch.save(model.state_dict(),"model.dth")
print(accuracy)

0.9884444444444445


In [95]:
#test_model
model.load_state_dict(torch.load("model.dth"))
model.eval()

results=list()
total=0

for itr,(image,label) in enumerate(test_dataloader):
    
    pred=model(image)
    pred=nn.functional.softmax(pred,dim=1)
    
    for i,p in enumerate(pred):
        if label[i]==torch.max(p.data,0)[1]:
            total=total+1
            results.append((image,torch.max(p.data,0)[1]))
test_accuracy=total/(itr+1)
print(accuracy)


0.9884444444444445
